In [ ]:
from pyvqnet.qnn.qlinear import QLinear
from pyvqnet.dtype import *
from pyvqnet.tensor.tensor import QTensor
from pyvqnet.tensor import tensor
import numpy as np
from sklearn.metrics import f1_score
import pandas as pd
from pyvqnet.data.data import data_generator
from pyvqnet.nn.module import Module
from pyvqnet.nn import Linear, ReLu
from pyvqnet.optim.adam import Adam
from pyvqnet.nn.loss import CrossEntropyLoss

from pyvqnet.qnn.pq3.quantumlayer import QuantumLayer
from pyvqnet.qnn.pq3.measure import ProbsMeasure
import pyqpanda3.core as pq 

from data_pre_v3_sy import data_preprocess
from pyvqnet.data.data import data_generator
from pyvqnet.tensor import QTensor
from pyqpanda3.quantum_info import StateVector


def load_data(train_csv, batch_size=32, shuffle: bool = True):
    """
    1. 读取数据并转换为张量
    """
    # 1. 数据预处理
    X_train, X_val, y_train, y_val, label_encoder, scaler = data_preprocess(train_csv)
    # X_train.shape = (3200, 9), X_val.shape = (800, 9)

    # 2. 转换为 VQNet 张量
    #    dtype 可选 Float32、Float64，取决于你的量子电路需求
    X_train_t = QTensor(X_train, dtype=kfloat32, requires_grad=False)
    y_train_t = QTensor(y_train.astype(np.int64), dtype=kfloat32, requires_grad=False)
    X_val_t   = QTensor(X_val,   dtype=kfloat32, requires_grad=False)
    y_val_t   = QTensor(y_val.astype(np.int64),   dtype=kfloat32, requires_grad=False)

    # # 3. 构建数据迭代器
    # train_loader = data_generator(X_train_t, y_train_t,
    #                               batch_size=batch_size,
    #                               shuffle=shuffle)
    # val_loader   = data_generator(X_val_t,   y_val_t,
    #                               batch_size=batch_size,
    #                               shuffle=False)

    # # 返回训练/验证迭代器，以及 label_encoder（用于解码预测结果）和 scaler（如果需要对测试集做同样的标准化）
    # return train_loader, val_loader, label_encoder, scaler
    
    return X_train_t, y_train_t, X_val_t, y_val_t, label_encoder, scaler

class VariationQuantumCircuit():
    """
    2. 在这里完成量子线路的设计
    """
    def __init__(self, num_qubits=9, depth=1):
        self.num_qubits = num_qubits
        self.depth      = depth
        # 量子模拟器后端
        self.machine = pq.CPUQVM()
        
    def __call__(self, x: np.ndarray) -> QTensor:
        """
        x:       一维特征向量，长度 = num_qubits
        """
        circ = pq.QCircuit()
        stv = StateVector(self.num_qubits)
        
        # —— 1. 特征映射——
        # (a) Hadamard 层：把所有 qubit 拉到叠加态
        for i in range(self.num_qubits):
            circ << pq.H(i)

        # (b) 单量子比特旋转：RZ(x_i)
        for i in range(self.num_qubits):
            circ << pq.RZ(i, x[i])
        
        # (c) 二阶交互项：对每一对 (i,j) 加入 CZ(x_i * x_j)
        for i in range((self.num_qubits-1)/2):
            for j in range(i+1, self.num_qubits):
                # 文章中 K_{ij} = (x_i * x_j) 交互
                theta = x[2*i] * x[2*j-1]
                # 这里把 CZ 当成 exp(-i theta Z_i Z_j / 2) 的近似
                circ << pq.RZZ(theta)
                
        # (c) 二阶交互项：对每一对 (i,j) 加入 CZ(x_i * x_j)
        for i in range((self.num_qubits-1)/2):
            for j in range(i+1, self.num_qubits):
                # 文章中 K_{ij} = (x_i * x_j) 交互
                theta = x[2*i+1] * x[2*j]
                # 这里把 CZ 当成 exp(-i theta Z_i Z_j / 2) 的近似
                circ << pq.RZZ(theta)# 固定 π 相位
        
        state = stv.update_by_evolve(circ)
        
        qstate_tensor = QTensor(state, dtype=kfloat32, requires_grad=False)
        
        return qstate_tensor       
        
    
    def calculate_K():
           return np.array([[1, 0], [0, 1]])


        

      
    
    
class AirQualityQuantumNN(Module):
    """
    3. 在这里完成初始化量子神经网络模型的代码
    """


def quantum_model_train():
    """
    4. 在这里完成训练量子模型的代码
    """
    X_train_t, y_train_t, X_val_t, y_val_t, label_encoder, scaler = load_data("train_data.csv")
    print(X_train_t, y_train_t, X_val_t, y_val_t)


def quantum_model_test():
    """
    5. 使用测试数据集验证模型
    """


if __name__ == "__main__":
    quantum_model_train()

    quantum_model_test()


== 数据基本信息 ==
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Temperature                    4000 non-null   float64
 1   Humidity                       4000 non-null   float64
 2   PM2.5                          4000 non-null   float64
 3   PM10                           4000 non-null   float64
 4   NO2                            4000 non-null   float64
 5   SO2                            4000 non-null   float64
 6   CO                             4000 non-null   float64
 7   Proximity_to_Industrial_Areas  4000 non-null   float64
 8   Population_Density             4000 non-null   int64  
 9   Air Quality                    4000 non-null   object 
dtypes: float64(8), int64(1), object(1)
memory usage: 312.6+ KB
None

== 数据统计信息 ==
                                count        mean         std     min  \
Temp